## RAG Medical Chatbot

### Setup

In [2]:
from dashboard.llm_functions import query_stream
from dashboard.chroma_functions import process_query
from pprint import pprint

In [3]:
import importlib

importlib.reload(importlib.import_module('dashboard.llm_functions'))
importlib.reload(importlib.import_module('dashboard.chroma_functions'))

<module 'dashboard.chroma_functions' from 'c:\\Users\\giuse\\Desktop\\RAGMedicalChatbot\\dashboard\\chroma_functions.py'>

### Generator Utility Function

In [33]:
# This function collects chunks from a generator, appends them to a global string, and yields each chunk.
def read_response(generator):
    output = []
    for chunk in generator:
        output.append(chunk)
    final_string = ''.join(output)
    return final_string

### Prompt Construction

In [5]:
SYS_PROMPT = 'Sei un chatbot medico italiano. Il tuo compito è assistere il paziente, comprendendo i suoi dubbi, fornendo una diagnosi e possibili trattamenti sulla base dei suoi sintomi. ' \
             'Se il paziente cita pochi sintomi non forzare la diagnosi. Non citare nella tua diagnosi altri dottori e altri casi di persone. Ignora i documenti non pertinenti. Generare testo entro le 380 words.'

In [6]:
current_query = [{'role': 'system', 'content': SYS_PROMPT},
                 {"role": "assistant", "content": "Come posso aiutarti? Non citare nella tua diagnosi altri dottori e altri casi di persone. Generare testo entro le 380 words."}
                ]

In [7]:
prompt = "ho mal di testa"

### RAG settings (Query Expansion, Summarization and Reranking)

In [38]:
# rag_prompt, urls = prompt, [] # NO RAG
# rag_prompt, urls = process_query(prompt, 'RAG', expansion=True, rerank=True, summarize=True) # RAG + Q + RR + S
rag_prompt, urls = process_query(prompt, 'RAG', expansion=True, rerank=True, summarize=False) # RAG + Q + RR

2024-10-22 18:53:23.365 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-22 18:53:23.366 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-22 18:53:23.367 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-22 18:53:23.369 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-22 18:53:23.369 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


DEBUG: Inizio processing
DEBUG: Query expansion completata
DEBUG: Query embedding completato
DEBUG: Retrieval completato
DEBUG: Reranking completato


In [60]:
# Print the expanded query to which the retrieved relevant documents will be attached.
pprint(rag_prompt[:610])

('I miei sintomi sono i seguenti: ho mal di testa, sto soffrendo di '
 "un'emicrania o di un'cefalea, un dolore sordo o pulsante nella parte "
 'superiore del cranio, forse localizzato nella fronte, nella parte posteriore '
 'della testa o nella zona occipitale, che può essere accompagnato da '
 'sensazione di tensione, di stanchezza o di stress, e che può essere '
 'influenzato da fattori come la luce, il rumore, il cibo o lo stress emotivo, '
 'e vorrei sapere se questo dolore è nuovo o se è aumentato di intensità negli '
 'ultimi tempi, e se è associato a altri sintomi come nausea, vomito, '
 'sensazione di debolezza o di vertigini..')


In [9]:
current_query.append({"role": "user", "content": rag_prompt})

### Answer Generation

In [61]:
print('DEBUG: Inizio generazione...')
generator = query_stream(current_query)
final_string = read_response(generator)
print('Risposta generata!')

DEBUG: Inizio generazione...
Risposta generata!


In [36]:
pprint(final_string)

('Grazie per avermi descritto i tuoi sintomi. Sembra che tu stia soffrendo di '
 'emicrania o cefalea, con un dolore sordo o pulsante nella parte superiore '
 'del cranio, localizzato nella fronte, nella parte posteriore della testa o '
 'nella zona occipitale. Il dolore può essere influenzato da fattori come la '
 'luce, il rumore, il cibo o lo stress emotivo.\n'
 '\n'
 'In base ai tuoi sintomi, non sembra che tu abbia una diagnosi di cefalea '
 'cronica esacerbata dallo sforzo e dalla manovra di Valsava in soggetto con '
 'malformazione di Chiari tipo 1, come descritto nel documento. Inoltre, non '
 'sembra che tu abbia una diagnosi di nevralgia trigeminale, come descritto '
 'nel documento.\n'
 '\n'
 'La diagnosi più probabile sembra essere emicrania o cefalea, con un dolore '
 'sordo o pulsante nella parte superiore del cranio. Tuttavia, per confermare '
 'la diagnosi e escludere altre possibili cause, ti consiglio di effettuare '
 'una visita presso un neurologo o un centro di cef

#### References used to answer

In [13]:
urls

['https://www.medicitalia.it/consulti/neurochirurgia/441534-emicrania-associata-a-sindrome-chiari-tipo-1.html',
 'https://www.medicitalia.it/consulti/terapia-del-dolore/860289-formicolii-scosse-non-dolorose-al-tatto.html',
 'https://www.dica33.it/esperto-risponde/domanda-mal-di-testa-58703.asp',
 'https://www.medicitalia.it/consulti/neurologia/850981-emicrania-vestibolare.html',
 'https://www.medicitalia.it/consulti/neurologia/748258-infiammazione-cronica-nervo-trigemino-5-mesi.html']

### Evaluation

In [ ]:
%pip install openpyxl

In [ ]:
%pip install rouge-score

In [17]:
import pandas as pd

# Replace 'your_file.xlsx' with the path to your Excel file
df = pd.read_excel('testmedicalchatbot.xlsx')

In [21]:
df.head(5)

,Query,SenzaRag,Rag+QueryExpansionReranking,Rag+QueryExpansionRerankingSummarize
0,ho mal di testa,Mi dispiace che tu stia soffrendo di mal di te...,Grazie mille per avermi condiviso i tuoi sinto...,Grazie per avermi descritto i tuoi sintomi. Da...
1,ho mal di stomaco con forti dolori addominali,Mi dispiace di sentire che hai mal di stomaco ...,Grazie per la descrizione dei tuoi sintomi. A ...,Grazie per avermi descritto i tuoi sintomi! Da...
2,sono giovane ma sto perdendo i capelli,Perdita di capelli in età giovanile! Non è un\...,Grazie per avermi condiviso i tuoi sintomi! Ec...,\nGrazie per avermi fornito i tuoi sintomi e p...
3,ho la febbre alta e la nausea da tre giorni,Mi dispiace di sentire che non stai bene. La\n...,La tua situazione sembra essere piuttosto\ncom...,Grazie per avermi fomito i sintomi e le inform...
4,ho la vista annebbiata durante il giorno,Interessante! La vista annebbiata durante il g...,"Basandomi sui sintomi che mi hai descritto, la...",Grazie per avermi fomito i sintomi che hai des...


In [20]:
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer
from rouge_score import rouge_scorer


# Function to calculate TF-IDF scores
def calculate_tfidf_scores(texts):
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(texts)
    feature_names = vectorizer.get_feature_names_out()
    return tfidf_matrix.toarray(), feature_names


# Function to calculate ROUGE-2 scores
def calculate_rouge(texts):
    # Here we include 'rouge2' instead of 'rouge1'
    scorer = rouge_scorer.RougeScorer(['rouge2', 'rougeL'], use_stemmer=True)
    rouge_scores = pd.DataFrame(index=['SenzaRag', 'Rag+QueryExpansionReranking', 'Rag+QueryExpansionRerankingSummarize'],
                                columns=['SenzaRag', 'Rag+QueryExpansionReranking', 'Rag+QueryExpansionRerankingSummarize'])

    for i in range(len(texts)):
        for j in range(len(texts)):
            if i != j:
                scores = scorer.score(texts[i], texts[j])
                rouge_scores.iloc[i, j] = scores['rougeL'].fmeasure  # Using F1 score for ROUGE-2

    return rouge_scores

# To aggregate the results
total_tfidf_scores = []
total_rouge_scores = []
total_difference_matrices = []
content_density_matrix = []

# Iterate over each row and compare text generations
for idx, row in df.iterrows():
    texts = [row['SenzaRag'], row['Rag+QueryExpansionReranking'], row['Rag+QueryExpansionRerankingSummarize']]

    # Calculate TF-IDF scores for the current row's texts
    tfidf_scores, feature_names = calculate_tfidf_scores(texts)

    # Calculate the sum of TF-IDF scores (informational content) for each text
    total_scores = tfidf_scores.sum(axis=1)

    # Calculate ROUGE-2 scores for the current row's texts
    rouge_scores = calculate_rouge(texts)

    # Calculate the Matrice di differenze informative
    difference_matrix = pd.DataFrame(index=['SenzaRag', 'Rag+QueryExpansionReranking', 'Rag+QueryExpansionRerankingSummarize'],
                                     columns=['SenzaRag', 'Rag+QueryExpansionReranking', 'Rag+QueryExpansionRerankingSummarize'])

    for i in range(len(total_scores)):
        for j in range(len(total_scores)):
            if i == j:
                difference_matrix.iloc[i, j] = 0  # Ignore comparison with itself

            elif i < j:
                # Use TF-IDF score difference and penalize it with ROUGE-2 score
                score_difference = total_scores[i] - total_scores[j]
                rouge_score = rouge_scores.iloc[i, j]
                difference_matrix.iloc[i, j] = (1 - rouge_score) * score_difference if isinstance(rouge_score, float) else score_difference
            else:
              difference_matrix.iloc[i, j] = 0  # Ignore comparison with itself

    # Append the results to lists for aggregation
    total_tfidf_scores.append(total_scores)
    total_rouge_scores.append(rouge_scores)
    total_difference_matrices.append(difference_matrix)

# Calculate the mean across all rows for each matrix
avg_tfidf_scores = np.mean(total_tfidf_scores, axis=0)
avg_rouge_scores = sum(total_rouge_scores) / len(total_rouge_scores)
avg_difference_matrix = sum(total_difference_matrices) / len(total_difference_matrices)

# Output the aggregated results
print("\nPunteggi totali TF-IDF per ogni testo:")
print(pd.DataFrame(avg_tfidf_scores, index=['SenzaRag', 'Rag+QueryExpansionReranking', 'Rag+QueryExpansionRerankingSummarize'], columns=["Punteggio TF-IDF"]))

print("\nMatrice di ROUGE-L (F1 score):")
print(avg_rouge_scores)

print("\nMatrice di differenze informative (comparazione delle generazioni di testo):")
print(avg_difference_matrix)


Punteggi totali TF-IDF per ogni testo:
                                      Punteggio TF-IDF
SenzaRag                                      9.209336
Rag+QueryExpansionReranking                  10.414443
Rag+QueryExpansionRerankingSummarize         10.726945

Matrice di ROUGE-L (F1 score):
                                      SenzaRag Rag+QueryExpansionReranking  \
SenzaRag                                   NaN                    0.137588   
Rag+QueryExpansionReranking           0.137588                         NaN   
Rag+QueryExpansionRerankingSummarize  0.133794                    0.247219   

                                     Rag+QueryExpansionRerankingSummarize  
SenzaRag                                                         0.133794  
Rag+QueryExpansionReranking                                      0.247219  
Rag+QueryExpansionRerankingSummarize                                  NaN  

Matrice di differenze informative (comparazione delle generazioni di testo):
             